In [1]:
import cv2
import yaml
from   FaceBoxes import FaceBoxes
from   TDDFA import TDDFA
from   utils.functions import draw_landmarks
import matplotlib.pyplot as plt
import imageio
from   utils.functions import cv_draw_landmark, get_suffix
import imageio_ffmpeg
import tqdm as tqdm
import numpy as np

from   tqdm import tqdm
from   network1 import IDreveal
from   numpy import load
import statistics as stat 

import os
import re
import argparse
import youtube_dl
import sys
import warnings
from   os.path import join
from   signal import signal, SIGINT, SIG_DFL
from   detection import test_full_image_network

c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
for i in range(0,30):

    video_fp_ = 'Dataset/Videos/00%{0}.mp4'.format(i)

    fn = video_fp_.split('/')[-1]
    reader = imageio.get_reader(video_fp_)
    fps = reader.get_meta_data()['fps']
    suffix = get_suffix(video_fp_)
    video_wfp = f'Dataset/Results{fn.replace(suffix, "")}_3d.mp4'
    writer = imageio.get_writer(video_wfp, fps=fps)

    # face detection
    face_boxes = FaceBoxes()

    # regress 3DMM params
    cfg = yaml.load(open('configs\mb1_120x120.yml'), Loader=yaml.SafeLoader)
    gpu_mode = cfg.get('gpu_mode', False)
    tddfa = TDDFA(gpu_mode = gpu_mode, **cfg)


    dense_flag = True
    pre_ver = None
    boxes = []
    boxes_b = []
    param_lst_b = []
    for j, frame in (enumerate(reader)):
        # print(frame)
        frame_bgr = frame[..., ::-1]  # RGB->BGR
        # print(frame_bgr)
        # if i == 0:
            # the first frame, detect face, here we only use the first face, you can change depending on your need
        boxes = face_boxes(frame_bgr)
        param_lst, roi_box_lst = tddfa(frame_bgr, boxes)
        boxes_b = boxes_b + boxes
        param_lst_b += param_lst
        np.savez_compressed('Dataset/feats/00{0}.npz'.format(i),boxes_b,param_lst_b)
    

In [2]:
device   = 'cpu' # in ('cpu', 'cuda:0', 'cuda:1')
time     = 100   # length of sequences in frames

In [3]:
net = IDreveal(time=time, device=device, weights_file='./model.th')
def extract_embedding(x):
    if isinstance(x, str):
        x = np.load(x) # load 3ddfa features
    y = net(x['arr_1']) # apply Temporal ID Network arr_1 contain the 3ddfa features array of vector have 62 coeff
    return y

In [ ]:
for i in (range(0,5)):


    ref_vids = [ 'Dataset/feats/reference/{0}'.format(i), ] # Reference Videos


    # extract embedded vectors for reference videos
    # print('Extracting embedded vectors for reference videos', flush=True)
    y = extract_embedding('./%s.npz' % str(ref_vids[0]))
    ref_embs = np.concatenate([extract_embedding('./%s.npz' % str(ref_vids[0]))], 0)
    print(flush=True)
    # print('Number of reference embedded vectors:', len(ref_embs), flush=True)


    list_dist = list()
    for j in tqdm(range(0,5)):    
        test_vids =  [ # Test Videos 
            #(Video, Type),
            'Dataset/feats/fake/{0}'.format(j),  1,
        ]

        typ_colors = ['C2', 'C3'] # color for each type of video
        typ_labels = ['Real videos', 'Deepfakes'] # label for each type of video





        # print('Extracting embedded vectors and distance computation for test videos', flush=True)
        
        count_embs = 0

        embs = extract_embedding('./%s.npz' % test_vids[0]) # extract embedded vectors for a test video
        count_embs = count_embs + len(embs)
        dist = np.min(np.sum(np.square(ref_embs[None,:,:]-embs[:,None,:]),-1),-1) # compute distances
        list_dist.append((dist, (test_vids[1])))
        #Jihad(To show the number of embeded vector for each video)
        #print('Total number of extracted vectors of video %s is:'%vid, len(embs), flush=True)
        ##########################

        print(flush=True)
        print('Total number of extracted vectors:', count_embs, flush=True)
    from cmath import sqrt
    x = sqrt(1.1)
    def set_color_violin(parts, color):
        #utlity function to change the color of violin-plot
        ret = None
        for keys in parts:
            if keys=='bodies':
                for pc in parts['bodies']:
                    ret = pc
                    pc.set_facecolor(color)
                    pc.set_edgecolor(color)
            else:
                parts[keys].set_edgecolor(color)
                parts[keys].set_facecolor(color)
        return ret
    
        
    # show violin-plot
    plt.figure(figsize=(15,8))
    typ_id = [None for _ in typ_labels]
    for i, (dist, typ) in enumerate(list_dist):
        parts = plt.violinplot(dist, positions=(i+0.5,) ,showmedians=True, points=dist.size)
        typ_id[typ] = set_color_violin(parts, typ_colors[typ])
    
    #Jihad 
    # print(list_dist)
    # #Jihad The threshold value is sqrt(1.1)
    # print(np.average(list_dist[7][0])) #mean value of distances for the video number 8 in testing videos
    # print(np.median(list_dist[7][0]))  #median value of distances for the video number 8 in testing videos
    
    plt.xlim([0,len(list_dist)])
    plt.xticks(np.arange(len(list_dist)),[])
    plt.ylim([0.5,8.5])
    plt.yticks([1,2,3,4,5,6,7,8,9,10,11],[], fontsize=10.0)
    plt.ylabel('Squared Euclidean distance', fontsize=14.0)
    plt.axhline(y = x.real + 0.5, color = 'b', linestyle = '-')
    plt.legend(typ_id,  typ_labels, bbox_to_anchor=(1.04, 1), loc='upper left', fontsize=12.0)
    plt.grid()
    plt.show()   



In [ ]:
from cmath import sqrt
x = sqrt(1.1)
TP = 0
TN = 0 
FP = 0 
FN = 0
print(list_dist)
print(len(list_dist))    
for i, (dist, typ) in enumerate(list_dist):
    median = np.median(list_dist[i][0])
    if (median < x.real) & typ == 0: 
        TP += 1
    elif (median > x.real) & typ == 1: 
        TN += 1
    elif (median < x.real) & typ == 1: 
        FP += 1
    elif (median > x.real) & typ == 0:
        FN += 1

acc = (TP + TN)/(TP + TN +FP + FN)
print(acc)

In [ ]:
model = 'models/x-model23.p'
import os
path_of_the_directory = 'E:\drivers'
ext = ('.mp4')
for files in os.listdir('Dataset/Videos/all/'):
    if files.endswith(ext):

        # video_path = files
        prediction = None
        prediction = test_full_image_network(video_path='Dataset/Videos/all/' + files,model_path= model, 
        start_frame= 0, output_path= "videoOut",fast='store_true')
        print("Prediction of it being fake in video{0}: ".format(files) + str(prediction["score"]))
        print("Output video in: " + prediction["file"]) 
    if files.__contains__('005'):
        break

In [ ]:
#ensemble model
from sklearn.ensemble import AdaBoostRegressor
estimators = []
estimators.append(('id_reveal', net))
estimators.append(('xception', model))

modelStack = AdaBoostRegressor(estimators)
# modelStack.fit(X_train, y_train)
# predictionStackingClass= modelStack.predict(X_test)
# print("Train Accuracy:",modelStack.score(X_train, y_train))
# print("Test Accuracy:",modelStack.score(X_test, y_test))
# print(classification_report(y_test,predictionStackingClass))